This notebook focuses on when to invest (timing optimization) based on
demand × marginal ROI, assuming similar seasonal patterns in FY2023.

In [ ]:
# === Notebook bootstrap: make repo root importable ===
import sys
from pathlib import Path

_cwd = Path.cwd().resolve()
for p in [_cwd, *_cwd.parents]:
    if (p / "src").exists():
        if str(p) not in sys.path:
            sys.path.insert(0, str(p))
        break

print("cwd:", _cwd)
print("sys.path[0]:", sys.path[0])


In [ ]:
# 01
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.io_utils import load_pickle, save_pickle, outputs_dir
from src.calendar_utils import make_fy_calendar

OUT_DIR = outputs_dir()
WS_PATH = Path(OUT_DIR) / "weekly_score.pkl"
ws = load_pickle(WS_PATH).copy()

TOP_N_BROADCAST = 10   # pulse
TOP_Q_ONLINE = 0.70    # top 30% weeks (threshold at 70% quantile)

FY_YEAR = 2023
# NOTE: Week starts on Sunday (aligned with raw CSV 'Week' column)


In [ ]:
# 02
print("ws columns:", ws.columns.tolist())
display(ws.head(3))

In [ ]:
# 03
# impact週の選定（week_of_yearベース）
ws["impact_score_online"] = ws["baseline_hat"] * ws["roi_online"].clip(lower=0)
ws["impact_score_broadcast"] = ws["baseline_hat"] * ws["roi_broadcast"].clip(lower=0)

# Broadcast：上位N
b_top = (
    ws.sort_values("impact_score_broadcast", ascending=False)
      .head(TOP_N_BROADCAST)
      .assign(media="Broadcast")
      [["week_of_year","media"]]
)

# Online：上位q%（=閾値以上）
thr_online = ws["impact_score_online"].quantile(TOP_Q_ONLINE)
o_top = (
    ws[ws["impact_score_online"] >= thr_online]
      .assign(media="Online")
      [["week_of_year","media"]]
)

impact_weeks = (
    pd.concat([b_top, o_top], ignore_index=True)
    .drop_duplicates()
    .assign(week_of_year=lambda d: d["week_of_year"].astype(int))
)

display(impact_weeks.sort_values(["media", "week_of_year"]))


In [ ]:
# 04
# FY2023の週カレンダーを生成（Sunday-start, FY=7/1）
fy_cal = make_fy_calendar(
    fy_year=FY_YEAR,
    fy_start_month=7,
    fy_start_day=1,
)

# 既存セルの "date" 列を使いたいので列名だけ合わせる
fy_cal = fy_cal.rename(columns={"week_start": "date"})

display(fy_cal.head(), fy_cal.tail())
print("fy_weeks:", fy_cal["fy_week"].min(), "-", fy_cal["fy_week"].max(), "count:", len(fy_cal))


In [ ]:
# 05
# カレンダーへ割り当て：week_of_year(=ISO週) → FYカレンダーの iso_week
impact_weeks_2023 = impact_weeks.merge(
    fy_cal[["date", "month", "iso_year", "iso_week"]],
    left_on="week_of_year",
    right_on="iso_week",
    how="left"
)

missing = impact_weeks_2023["date"].isna().sum()
print("missing mapped weeks:", missing)

# FY_YEARに存在しない ISO週（例：53週）は日付にマップできないため除外
impact_weeks_2023 = impact_weeks_2023.dropna(subset=["date"]).copy()

display(impact_weeks_2023.sort_values(["media", "week_of_year"]))

In [ ]:
# 06
# calendar_df 作成（Broadcast / Online / both）
calendar_df = (
    impact_weeks_2023
    .pivot_table(
        index=["date","month"],
        columns="media",
        values="week_of_year",
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)

calendar_df["both"] = (calendar_df.get("Broadcast", 0) > 0) & (calendar_df.get("Online", 0) > 0)
display(calendar_df.head(20))


In [ ]:
# 07
# 可視化
fig, ax = plt.subplots(figsize=(14, 3))

for _, row in calendar_df.iterrows():
    if row.get("Broadcast", 0) and row.get("Online", 0):
        color = "#A6CE25"   # both
    elif row.get("Broadcast", 0):
        color = "#4DA9CD"   # Broadcast
    elif row.get("Online", 0):
        color = "#8453F6"   # Online
    else:
        continue

    ax.bar(row["date"], 1, width=5, color=color)

ax.set_title(f"FY{FY_YEAR} Impact Weeks (Broadcast / Online) - FY calendar (Sunday-start)")
ax.set_yticks([])
ax.set_xlabel("Date")
plt.tight_layout()
plt.show()


### Interpretation:
- Based on the integrated evaluation of demand and marginal ROI,Broadcast, investments tend to concentrate in spring (April–June), where they are effective for building broad awareness.
- In contrast, Online investments show higher effectiveness in autumn (September–November), when baseline demand is elevated and conversion efficiency remains high.

These results suggest a complementary investment structure:
 Broadcast plays a leading role in awareness formation during spring, while Online media supports demand capture during the high-demand period in autumn.